# Learning and Decision Making

## Laboratory 4: Partially observable Markov decision problems

In the end of the lab, you should submit all code/answers written in the tasks marked as "Activity n. XXX", together with the corresponding outputs and any replies to specific questions posed to the e-mail <adi.tecnico@gmail.com>. Make sure that the subject is of the form [&lt;group n.&gt;] LAB &lt;lab n.&gt;.

### 1. Modeling

Consider once again the guessing game domain described in the Homework and which you described as a POMDP.

Recall that:

* The opponent can hold one of two cards in hand: an Ace of Clubs (A&clubs;) and an Ace of Diamonds (A&diams;). The agent must guess which card the opponent is holding. 

* For every right answer, the agent wins 1EUR, and every wrong answer costs the agent 1EUR. 

* The agent can also try to _peek_. 

* When the agent peeks, it sees the right card with a probability of 0.9 and the wrong card with probability 0.1.

* The game restarts whenever the agent makes a guess.

Consider throughout that $\gamma=0.9$.

---

#### Activity 1.        

Implement your POMDP in Python. In particular,

* Create a list with all the states;
* Create a list with all the actions;
* Create a list with all the observations
* For each action, define a `numpy` array with the corresponding transition probabilities;
* For each action, define a `numpy` array with the corresponding observation probabilities;
* Define a `numpy`array with the cost that you defined in your homework.

The order for the states and actions used in the transition probability and cost matrices should match that in the lists of states and actions. 

**Note**: Don't forget to import `numpy`.

---

In [36]:
# Insert your code here
import numpy as np

states = ["Clubs" , "Diamonds"]

actions = ["SelectClubs", "SelectDiamonds", "Peek"]

observations = ["Clubs", "Diamonds"]

P_Peek = np.array([[1, 0], [0, 1]])

P_Clubs = np.array([[0.5, 0.5], [0.5, 0.5]])

P_Diamonds = np.array([[0.5, 0.5], [0.5, 0.5]])


O_Peek = np.array([[0.9, 0.1], [0.1, 0.9]])

O_Clubs = np.array([[0.5, 0.5], [0.5, 0.5]])

O_Diamonds = np.array([[0.5, 0.5], [0.5, 0.5]])

Cost = np.array([[0, 1, 0.1], [1, 0, 0.1]])



### 2. Sampling

You are now going to sample random trajectories of your POMDP and observe the impact it has on the corresponding belief.

---

#### Activity 2.

Generate a random POMDP trajectory using a uniformly random policy. In particular, from a random initial state $x_0$ generate:

1. A sequence of 10,000 states by selecting the actions uniformly at random;
2. The corresponding sequence of 10,000 actions;
3. The corresponding sequence of 10,000 observations.

---

In [45]:
# Insert your code here

import numpy as np

t = 0

sequence_states = []
sequence_actions = []
sequence_observations = []

while t<25:
    
    state = np.random.choice(states,1, replace=False)
    action = np.random.choice(actions,1, replace=False)
    
    if state == "Clubs":
        observation_probabilities = O_Peek[:,0] # [0.9]
                                                # [0.1]
    else:
        observation_probabilities = O_Peek[:,1] # [0.1]
                                                # [0.9]
    if action == "Peek":
        observation = np.random.choice(observations,1, replace=False, p=observation_probabilities)
    else:
        observation = state # se a accao for adivinhar uma carta a nossa observacao
                            # vai ser igual ao estado (que e a carta que esta na mao do oponente)

    sequence_states += [state]
    sequence_actions += [action]
    sequence_observations += [observation]
    t+=1
    
print "STATES(carta que esta na mao do oponente)\n"
for i in sequence_states:
    print i
print "\nACTIONS(accao que o agente escolhe)\n"    
for i in sequence_actions:
    print i
print "\nOBSERVATIONS(observacao que o agente faz)\n"
for i in sequence_observations:
    print i

STATES(carta que esta na mao do oponente)

['Diamonds']
['Diamonds']
['Clubs']
['Clubs']
['Clubs']
['Clubs']
['Clubs']
['Clubs']
['Clubs']
['Diamonds']
['Clubs']
['Diamonds']
['Clubs']
['Diamonds']
['Clubs']
['Clubs']
['Diamonds']
['Clubs']
['Diamonds']
['Diamonds']
['Clubs']
['Diamonds']
['Diamonds']
['Clubs']
['Clubs']

ACTIONS(accao que o agente escolhe)

['Peek']
['SelectDiamonds']
['SelectDiamonds']
['Peek']
['SelectDiamonds']
['SelectClubs']
['Peek']
['SelectClubs']
['SelectDiamonds']
['SelectClubs']
['SelectDiamonds']
['SelectDiamonds']
['SelectDiamonds']
['SelectDiamonds']
['SelectDiamonds']
['SelectClubs']
['SelectDiamonds']
['Peek']
['SelectClubs']
['SelectClubs']
['SelectClubs']
['SelectClubs']
['SelectClubs']
['Peek']
['Peek']

OBSERVATIONS(observacao que o agente faz)

['Diamonds']
['Diamonds']
['Clubs']
['Clubs']
['Clubs']
['Clubs']
['Clubs']
['Clubs']
['Clubs']
['Diamonds']
['Clubs']
['Diamonds']
['Clubs']
['Diamonds']
['Clubs']
['Clubs']
['Diamonds']
['Clubs']
['Diamond

---

#### Activity 3.

For the POMDP trajectory generated in Activity 2, compute the corresponding sequence of beliefs, assuming that the initial belief is $\mathbf{b}_0=[0.5, 0.5]$. Report the resulting beliefs, ignoring duplicate beliefs or beliefs whose distance is smaller than $10^{-4}$.

**Note 1:** You may want to define a function `belief_update` that receives a belief, an action and an observation and returns the updated belief.

**Note 2:** To compute the distance between vectors, you may find useful `numpy`'s function `linalg.norm`.


---

In [81]:
# Insert your code here


initial_belief = np.array([[0.7, 0.3]]) # mudar para [0.5, 0.5]

def belief_update(belief, action, observation):
        
    if action == "Peek":
        
        Pa = P_Peek
        
        if observation == "Clubs":
            Diag_O = np.diag(O_Peek[:,0])
        else:
            Diag_O = np.diag(O_Peek[:,1])
            
    elif action == "SelectClubs" or action =="SelectDiamonds":
        
        Pa = P_Clubs # Clubs e Diamonds tem P iguais por isso nao faz diferenca
        
        if observation == "Clubs":
            Diag_O = np.diag(O_Peek[:,0])
        else:
            Diag_O = np.diag(O_Peek[:,1])

    
    aux1 = np.dot(belief, Pa * Diag_O) # belief * Pa * Diag_O
    aux2 = aux1[:,0] + aux1[:,1] # [a, b] --> a+b

    
    updated_belief = aux1/aux2
    
    return updated_belief
 
print belief_update(initial_belief,"Peek","Diamonds")

[[ 0.20588235  0.79411765]]


### 3. Solution methods

In this section you are going to compare different non-exact solution methods.

---

#### Activity 4

Compute the solution for the underlying MDP and report the corresponding optimal policy and optimal cost-to-go. 

** Note:** You may reuse code from previous labs.

---

In [68]:
# Insert your code here

# TESTING RANDOM STUFF
test = np.array([[1, 3]])

aux = test[:,0]+test[:,1]

result = test/np.linalg.norm(test)

print result

print np.linalg.norm(test)




[[ 0.31622777  0.9486833 ]]
3.16227766017


---

#### Activity 5

For each of the beliefs computed in Activity 3, compute the action prescribed by:

* The MLS heuristic;
* The AV heuristic;
* The Q-MDP heuristic.

---

In [ ]:
# Insert your code here

---

#### Activity 6

Suppose that the optimal cost-to-go function for the POMDP can be represented using the $\alpha$-vectors

$$
\left\{
\begin{bmatrix}
2.795\\
3.795
\end{bmatrix},
\begin{bmatrix}
3.795\\
2.795
\end{bmatrix},
\begin{bmatrix}
3.105\\
3.105
\end{bmatrix}\right\}$$

corresponding to the actions 'Guess clubs', 'Guess diamonds' and 'Peek', respectively. Represent the optimal cost-to-go function and compare the optimal policy with the MDP heuristics from Activity 5 in the beliefs computed in Activity 3.

** Note: ** Don't forget to import `matplotlib`, and use the magic `%matplotlib notebook`.

---

In [ ]:
# Insert your code here